1. get sample image
2. download image
3. perform imnage grouping
4. inspect result

Next items to do, prepare and package as jobs
1. create auto-query
2. create sql to store result
3. export to executable py


# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

# prepare data for saving to rdbms
# print(clus_new)
# i = 0 
result_cluster = []

for item in clus_new:
    # print(clus_new[item])
    print(len(clus_new[item]))
    for file in clus_new[item]:
        # print(file, item)
        result_cluster.append(file)
    # i = i + 1

# view_cluster_2(0)
len(clus_new)
len(result_cluster)


 * Kesepakatan status di kolom screen_analisis_ai.status
 * 1 --> baru diinput
 * 2 --> lagi dikerjakan
 * 3 --> proses berhasil
 * 4 --> proses gagal
 *
 
 * Kesepakatan jenis analisa AI
 * 1 --> Analisa Cluster
 * 2 --> Analisa image clustering
 * 3 --> Analisa sentiment


In [1]:

# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 
from sklearn.metrics.pairwise import cosine_similarity

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import data_connector

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import requests
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse

2024-06-20 13:01:28.150026: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 13:01:28.827478: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
strSQL = """
select 	a.id as jobsid,
        a.*,
		c.id as key_monitoring_media_social, 
		d.id as key_monitoring_media_online,
		c.*
from 	screen_analisis_ai a 
		inner join monitoring_search b
			on cast(a.monitoring_id as varchar) = cast(b.id as varchar)
		inner join monitoring_media_social c
			on b.id = c.monitoring_search_id 
		left outer join  monitoring_media_online d
			on d.monitoring_search_id = c.monitoring_search_id 
where a.jenis_analisa = '20'
and a.status = 1
order by a.created desc 
limit 1
"""

df_job = data_connector.execute_query_psql(strSQL)
if len(df_job) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
similarity_treshold = 0.9
i_process_id = df_job['jobsid'][0]
screen_name = ''
database_keyword_id = df_job['key_monitoring_media_social'][0]
social_media_monitoring_id = df_job['key_monitoring_media_social'][0]
media_online_monitoring_id = df_job['key_monitoring_media_online'][0]

# print(database_keyword_id)
print(similarity_treshold)
print(i_process_id)
print(social_media_monitoring_id)
print(media_online_monitoring_id)


# Prepare SQL Statement
print(i_process_id)
sql = "update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = %s"
sql = sql.replace('%s', str(i_process_id))

print(sql)
row_count = data_connector.execute_query_psql(sql)
print('update ' + str(row_count) + ' rows')

# Create Header Record
sql = "insert into ret_analysis_header (job_id, datetime_start, user_id) values (%s, now(), %s)" % (i_process_id,"1")
# Execute the query
data_connector.execute_query_psql(sql)



0.9
5068
e8e75fb1-ff08-40ee-9517-533ce69d1e4a
None
5068
update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = 5068
update 1 rows


1

In [3]:
## Create as one function here 
def resize_and_upload_image(url, save_folder):
    try:
        # Fetch the image from the URL
        response = requests.get(url)
        response.raise_for_status()

        # Open the image from the response content
        image = Image.open(BytesIO(response.content))

        # Check the image size in bytes
        image_size = len(response.content)

        # Set the maximum allowed size in bytes (200KB)
        max_size = 200 * 1024

        # Extract the base path from the URL for saving the images
        parsed_url = urlparse(url)
        base_path = os.path.basename(parsed_url.path)

        # Determine the paths for saving the images
        original_image_path = os.path.join(save_folder, 'original_' + base_path)
        resized_image_path = os.path.join(save_folder, 'resized_' + base_path)

        # If the image size exceeds the maximum allowed size, resize it
        if image_size > max_size:
            print('more than 200kb')
            print(original_image_path)
            print(resized_image_path)
            
            # Calculate the scaling factor to reduce the image size
            scale_factor = (max_size / image_size) ** 0.5
            new_width = int(image.width * scale_factor)
            new_height = int(image.height * scale_factor)

            # Resize the image
            resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)

            # Save the resized image to the local folder
            resized_image.save(resized_image_path, format="JPEG")

            # Reupload the resized image to the original location
            with open(resized_image_path, 'rb') as resized_file:
                reupload_url = url  # Use the original URL for reupload
                print(url)
                files = {'file': ('resized_' + base_path, resized_file)}
                reupload_response = requests.post(reupload_url, files=files)
                print(reupload_response)

            # Delete the original image in the local folder
            os.remove(original_image_path)

        else:
            # Image is smaller than 200KB, no need to resize or reupload
            resized_image_path = original_image_path

        return original_image_path, resized_image_path

    except Exception as e:
        # Handle errors such as invalid URLs or image processing issues
        return str(e)

# Example usage:
# imageURL = "https://example.com/path/to/your/image.jpg"
# save_folder = "path/to/your/local/folder"
# original_image_path, resized_image_path = resize_and_upload_image(imageURL, save_folder)
# Both original_image_path and resized_image_path will contain the file paths in the local folder.

def download_image(url, folder_path):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Extract the filename from the URL
            filename = os.path.join(folder_path, os.path.basename(url))

            # Save the image to the specified folder location
            with open(filename, 'wb') as file:
                file.write(response.content)
            file_size_bytes = os.path.getsize(filename)
            file_size_kb = file_size_bytes / 1024

            if file_size_kb >= 200:
                print("Resizing...")
                resize_and_upload_image(filename,url)
            # print(f"Image downloaded and saved as {filename}")
        else:
            print(f"Failed to download image. Status code: {response.status_code}")

    except Exception as e:
        print(f"Error: {str(e)}")

def resize_and_upload_image(file_path, upload_url, target_size_kb=200):
    try:
        # Open the image from the file
        with Image.open(file_path) as image:
            # Calculate the target size in bytes
            target_size_bytes = target_size_kb * 1024

            # Initialize the quality variable
            quality = 95

            while os.path.getsize(file_path) > target_size_bytes:
                # Resize the image while keeping the quality constant
                width, height = image.size
                new_width = int(width * 0.9)
                new_height = int(height * 0.9)

                # Ensure the dimensions are at least 1
                new_width = max(1, new_width)
                new_height = max(1, new_height)

                image = image.resize((new_width, new_height), Image.LANCZOS)

                # Save the resized image to the same file
                image.save(file_path, "JPEG", quality=quality)

            if os.path.getsize(file_path) <= target_size_bytes:
                print(f"Image resized and overwritten at: {file_path}")

                # Upload the resized image to the given URL
                with open(file_path, 'rb') as file:
                    response = requests.post(upload_url, files={'file': (os.path.basename(file_path), file)})

                if response.status_code == 200:
                    print(f"Image uploaded to {upload_url}")
                else:
                    print(f"Upload failed with status code {response.status_code}")
            else:
                print(f"Image not resized, not uploaded.")
                
    except Exception as e:
        print(f"Error: {e}")

## query functions
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.util import deprecations
from sqlalchemy.sql import text
deprecations.SILENCE_UBER_WARNING = True

def execute_query_psql(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

def clear_image():
    folder_path = '/home/ubuntu/ml_project/notebook_directory/multipool/ai-case-study/img'
    # Create a Path object for the folder
    folder = Path(folder_path)
    
    # Iterate through the files in the folder and delete them
    for file in folder.iterdir():
        try:
            if file.is_file():
                file.unlink()
                # print(f"Deleted {file}")
            else:
                print(f"{file} is not a file.")
        except Exception as e:
            print(f"Error deleting {file}: {str(e)}")

# extracting feature from image files
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx) #, use_multiprocessing=True)
    return features

# function that lets you view a cluster (based on identifier)        
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

In [4]:
# 10 = tiktok
# 20 = youtube
# 30 = instagram_post
# 40 = facebook_post
# 50 = google_result

def process_image_clustering(social_media_type, social_media_monitoring_id):
    print('starting function')

    ## tiktok
    if(social_media_type == 10):
        query = """
        select 	a.id as key_id,
        		b.filename as filename
        from 	tiktok a inner join media_files_tiktok b 
        		on a.id = b.tiktok_id 
        where a.monitoring_id = '%s'
        """
        social_media_type_desc = "tiktok"

    ## Youtube
    elif(social_media_type == 20):
        query = """
        select 	* 
        from 	facebook_post a inner join media_files_facebook b
        		on a.id = b.facebook_id 
        where 	monitoring_id = '%s'
        """
        social_media_type_desc = "facebook"    

    ## instagram
    elif(social_media_type == 30):
        query = """
        select 	* 
        from 	facebook_post a inner join media_files_facebook b
        		on a.id = b.facebook_id 
        where 	monitoring_id = '%s'
        """
        social_media_type_desc = "facebook"

    ## facebook
    elif(social_media_type == 40):
        query = """
        select 	* 
        from 	facebook_post a inner join media_files_facebook b
        		on a.id = b.facebook_id 
        where 	monitoring_id = '%s'
        """
        social_media_type_desc = "facebook"

    ## google result
    elif(social_media_type == 50):
        query = """
        select 	* 
        from 	facebook_post a inner join media_files_facebook b
        		on a.id = b.facebook_id 
        where 	monitoring_id = '%s'
        """
        social_media_type_desc = "facebook"
    else:
        return
        
    query = query % str(social_media_monitoring_id)
    print(query)
    
    result_df = data_connector.execute_query_psql(query)
    
    # 3. check, minimum 100 images
    if len(result_df) < 100:
        print('minimum sample size is 100, stop processing ' + str(len(result_df)))
        return
        
    ## downloading image set
    # https://disestages.com/api/media/photo/download/facebook/1262503305067058.jpg
    img_prefix_http = "https://disestages.com/api/media/photo/download/" + social_media_type_desc + "/"
    folder_path = "/home/ubuntu/ml_project/notebook_directory/multipool/ai-case-study/img"    # Replace with the desired folder path
    
    # download_image(url, folder_path)
    # clear up folder first
    # clear_image()
    
    i = 0
    for index, row in result_df.iterrows():
        # print(result_df.at[index,'filename'])
        url = img_prefix_http + result_df.at[index,'filename']
        print('downloading :' + url)
        download_image(url, folder_path)
        i = i +1
    
    print("finished downloading " + str(i) + " image set" )
    
    path = folder_path
    # change the working directory to the path where the images are located
    os.chdir(path)
    
    # this list holds all the image filename
    flowers = []
    
    # creates a ScandirIterator aliased as files
    with os.scandir(path) as files:
      # loops through each file in the directory
        for file in files:
            if file.name.endswith('.jpg'):
              # adds only the image files to the flowers list
                flowers.append(file.name)
            if file.name.endswith('.png'):
                # adds only the image files to the flowers list
                flowers.append(file.name)
                
    model = VGG16()
    model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
    
    data = {}
    
    # loop through each image in the dataset
    for flower in flowers:
        # try to extract the features and update the dictionary
        try:
            #print(flower)     
            feat = extract_features(flower,model)
            data[flower] = feat
            # print(feat)
        # if something fails, save the extracted features as a pickle file (optional)
        except:
            with open(folder_path + '/tmp.pkl','wb') as file:
                pickle.dump(data,file)
    
    # get a list of the filenames
    filenames = np.array(list(data.keys()))
    
    # get a list of just the features
    feat = np.array(list(data.values()))
    
    # reshape so that there are 210 samples of 4096 vectors
    feat = feat.reshape(-1,4096)
    
    # get the unique labels (from the flower_labels.csv)
    df = pd.read_csv('/home/ubuntu/ml_project/notebook_directory/multipool/ai-case-study/flower_labels.csv')
    label = df['label'].tolist()
    unique_labels = list(set(label))
    
    # reduce the amount of dimensions in the feature vector
    pca = PCA(n_components=100, random_state=22)
    pca.fit(feat)
    x = pca.transform(feat)
    
    # cluster feature vectors
    kmeans = KMeans(n_clusters=len(unique_labels), random_state=22)
    kmeans.fit(x)
    
    # holds the cluster id and the images { id: [images] }
    groups = {}
    for file, cluster in zip(filenames,kmeans.labels_):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)
    
    # this is just incase you want to see which value for k might be the best 
    sse = []
    list_k = list(range(3, 50))
    
    for k in list_k:
        km = KMeans(n_clusters=k, random_state=22)
        km.fit(x)
        
        sse.append(km.inertia_)
    
    # function to calculate clusters
    def cluster(filePaths, features, threshold=0.9):
        features = features.reshape(-1,4096)
        simMatrix = cosine_similarity(features)
        clusters = {}
        for i in range(len(features)):
            dupIdx = list(np.where(simMatrix[i] > threshold)[0])
            # The similarity matrix will include comparisons of items with themselves, which will always 
            # result in a similarity of 1.0 (100%) and is redundant, so we ignore those
            if len(dupIdx) > 1:
                curCluster, clusterMatch = list(dupIdx), None
                # The first time an image is found to be in any given cluster, we log the entire cluster, 
                # so subsequent checks of other images from the same cluster would result in duplicated clusters.
                # Check for that here
                for cIdx in clusters:
                    if curCluster[0] in clusters[cIdx]:
                        clusterMatch = cIdx
                        break
                # If the current cluster didn't match any existing ones, create/log it
                if clusterMatch == None: clusters[len(clusters)] = curCluster
        # Resolve file indices back to file paths
        for cIdx in clusters: clusters[cIdx] = [filePaths[x] for x in clusters[cIdx]]
        return clusters
    
    # another method of clustering based on CSI
    clus_new = cluster(filenames, feat, threshold=0.6)
    
    # prepare data for saving to rdbms
    save_df = result_df
    
    # save_df[["file_folder","filename_actual"]] = save_df['filename'].str.split('/',n=1, expand=True)
    save_df['cluster_number'] = 0
    
    for item in clus_new:
        # print(clus_new[item])
        # print(len(clus_new[item]))
        for file in clus_new[item]:
            # print(item)
            # print(file)
            filter_condition = save_df['filename'] == file
            save_df.loc[filter_condition,'cluster_number'] = item
            result = save_df[filter_condition]
            # result['cluster_number'] = item
            # print(save_df.loc[filter_condition,'cluster_number'])
    
    ## saving result to table
    # jobid, tweet_id, cluster_no
    
    # 10 = tiktok
    # 20 = youtube
    # 30 = instagram_post
    # 40 = facebook_post
    # 50 = google_result
    
    
    s_cluster_number = ""
    
    for index, row in save_df.iterrows():
        if row['cluster_number'] == '':
            s_cluster_number = "NULL"
        else:
            s_cluster_number = row['cluster_number']
            
        sql = "INSERT into ret_cluster_image_result_monitor (ref_id, cluster_no, platform_id, job_id) values (%s, %s, %s, %s)" % (row['key_id'], s_cluster_number, "10", str(i_process_id))
        
        # print(sql)
        data_connector.execute_query_psql(sql)

In [5]:
## saving result to table
# jobid, tweet_id, cluster_no

# 10 = tiktok
# 20 = youtube
# 30 = instagram_post
# 40 = facebook_post
# 50 = google_result

if df_job['is_tiktok'][0]:
    print("processing tiktok")
    process_image_clustering(10, social_media_monitoring_id)

# if df_job['is_youtube'][0]:
#     print("processing youtube")
#     process_image_clustering(20, social_media_monitoring_id)

# if df_job['is_instagram'][0]:
#     print("processing instagram")
#     process_image_clustering(30, social_media_monitoring_id)

# if df_job['is_facebook'][0]:
#     print("processing facebook")
#     process_image_clustering(40, social_media_monitoring_id)

# if df_job['is_google'][0]:
#     print("processing google")
#     process_image_clustering(50, media_online_monitoring_id)

# closing .... report back job status into rdbms
sql = "update screen_analisis_ai set end_process = now(), status = 3, processby_id = 1 where id = %s" % (str(i_process_id))
data_connector.execute_query_psql(sql)

sql = "update screen_analisis_ai set duration = EXTRACT(EPOCH FROM (end_process - start_process)) where id = " + str(i_process_id)
data_connector.execute_query_psql(sql)

processing tiktok
starting function

        select 	a.id as key_id,
        		b.filename as filename
        from 	tiktok a inner join media_files_tiktok b 
        		on a.id = b.tiktok_id 
        where a.monitoring_id = 'e8e75fb1-ff08-40ee-9517-533ce69d1e4a'
        
downloading :https://disestages.com/api/media/photo/download/tiktok/7336101392790162693.jpg
downloading :https://disestages.com/api/media/photo/download/tiktok/7289005334667021574.jpg
downloading :https://disestages.com/api/media/photo/download/tiktok/7335372089085480210.jpg
downloading :https://disestages.com/api/media/photo/download/tiktok/7332816677832264966.jpg
downloading :https://disestages.com/api/media/photo/download/tiktok/7336487840475352326.jpg
downloading :https://disestages.com/api/media/photo/download/tiktok/7336046086227774721.jpg
downloading :https://disestages.com/api/media/photo/download/tiktok/7325743936788548882.jpg
downloading :https://disestages.com/api/media/photo/download/tiktok/73368076223827182

2024-06-20 13:01:37.703381: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 13:01:37.732439: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 13:01:37.733858: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


I0000 00:00:1718888500.472047  410987 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━

1

In [6]:
# wait 10 seconds before finished
import time
time.sleep(10)